<a href="https://colab.research.google.com/github/Pratik2222/Projects/blob/main/Fake_News_Classifier_Using_LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [5]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [6]:
#Drop Nan values from a row
df = df.dropna()

In [7]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [8]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [9]:
X.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [10]:
## Get the Dependent features
y=df['label']
y.head(10)

0     1
1     0
2     1
3     1
4     1
5     0
7     0
9     0
10    0
11    0
Name: label, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.8.2'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

In [15]:
messages=X.copy()

In [16]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()   
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3079, 269, 3023, 4525, 998, 1081, 3621, 4404, 4256, 4263],
 [4109, 485, 4680, 2401, 2300, 652, 1057],
 [2721, 1596, 2366, 749],
 [2219, 956, 1481, 3522, 126, 4765],
 [4223, 2300, 4206, 3844, 1957, 448, 2300, 3692, 100, 3940],
 [4314,
  4387,
  3213,
  1073,
  3972,
  2294,
  2084,
  2921,
  3366,
  2323,
  3744,
  349,
  4627,
  1651,
  1057],
 [3541, 591, 1597, 1415, 2253, 1441, 4776, 954, 3675, 1848, 3841],
 [3927, 1044, 701, 4791, 1055, 4644, 2294, 775, 3675, 1848, 3841],
 [1396, 725, 799, 2896, 4274, 2830, 1415, 615, 2294, 2755],
 [1433, 4595, 2768, 949, 3547, 3643, 3485, 4266],
 [2382, 1928, 4393, 3120, 477, 3619, 1456, 889, 1845, 1451, 4371],
 [3522, 3109, 998, 2830, 2294, 1055],
 [4516, 474, 2261, 3532, 4058, 2262, 3472, 2342, 1479],
 [4882, 4913, 1337, 733, 3456, 1271, 66, 3675, 1848, 3841],
 [2533, 145, 4804, 171, 4879, 3675, 1848, 3841],
 [1233, 4239, 2704, 1373, 1258, 1972, 304, 2795, 4713, 3766],
 [846, 1860, 485],
 [4263, 705, 2696, 1896, 2294, 3339, 1599, 1057],
 [90, 1

In [25]:
#Embedding Representation
sent_length=20
#Applying padding, to make the sentences/messages length same in embedding matrix
#In this case 20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4404 4256 4263]
 [   0    0    0 ... 2300  652 1057]
 [   0    0    0 ... 1596 2366  749]
 ...
 [   0    0    0 ... 3675 1848 3841]
 [   0    0    0 ... 4950 1973 3519]
 [   0    0    0 ...  894 2248 4066]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3079,
        269, 3023, 4525,  998, 1081, 3621, 4404, 4256, 4263], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 9ms/step - loss: 0.3420 - accuracy: 0.8430 - val_loss: 0.1998 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1460 - accuracy: 0.9434 - val_loss: 0.1998 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0982 - accuracy: 0.9620 - val_loss: 0.2149 - val_accuracy: 0.9133
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0683 - accuracy: 0.9758 - val_loss: 0.2724 - val_accuracy: 0.9109
Epoch 5/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0387 - accuracy: 0.9873 - val_loss: 0.3181 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0227 - accuracy: 0.9922 - val_loss: 0.3849 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0141 - accuracy: 0.9958 - val_loss: 0.4814 - val_accuracy: 0.9167
Epoch 

In [31]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3094,  325],
       [ 190, 2426]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9146644573322287

In [33]:
from tensorflow.keras.layers import Dropout
## Creating model with drop out layer
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
import numpy as np
X_final1=np.array(embedded_docs)
y_final1=np.array(y)

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_final1, y_final1, test_size=0.33, random_state=42)

### Finally Training
model.fit(X_train1,y_train1,validation_data=(X_test1,y_test1),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 9ms/step - loss: 0.3433 - accuracy: 0.8358 - val_loss: 0.1915 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1488 - accuracy: 0.9404 - val_loss: 0.1885 - val_accuracy: 0.9203
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1068 - accuracy: 0.9594 - val_loss: 0.2094 - val_accuracy: 0.9223
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0854 - accuracy: 0.9692 - val_loss: 0.2166 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0713 - accuracy: 0.9757 - val_loss: 0.2541 - val_accuracy: 0.9077
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0589 - accuracy: 0.9799 - val_loss: 0.2931 - val_accuracy: 0.9077
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0429 - accuracy: 0.9854 - val_loss: 0.3190 - val_accuracy: 0.9104
Epo

In [35]:
y_pred1=(model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test1,y_pred1)

array([[3138,  281],
       [ 256, 2360]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test1,y_pred1)

0.9110190555095278